In [57]:
import h5py
import os
import pandas as pd
import json
import numpy as np

# Vorbereitungen

## Dateien in Array schreiben

In [66]:
directory = 'datasetsRosen'
datasets = []

for filename in os.listdir(directory):
    file = h5py.File(directory + '/' + filename)
    for key in file.keys():
        datasets.append(file[key])
        
print(f'{len(datasets)} Datensätze erfolgreich in datasets-Array eingelesen')

2000 Datensätze erfolgreich in datasets-Array eingelesen


## Dateien in Pandas Dataframe schreiben (und velocity berechnen)

In [142]:
def read_hdf5(file_path):
    """Read an HDF5 file into a Pandas DataFrame and return it along with attributes."""
    with h5py.File(file_path, 'r') as file:
        for keyname in file.keys():
            if keyname != "":
                data_group_name = keyname
            else:
                print(f"'data' missing in: {file_path}")
                return None, None

        data_group = file[data_group_name]
        data_dict = {name.lower().rstrip('_'): data_group[name][:] for name in data_group.keys()}

        # Extract attributes
        attributes = {attr: data_group.attrs[attr] for attr in data_group.attrs.keys()}
        data_id = attributes.get('id', 'Unknown')

        max_length = max(len(values) for values in data_dict.values())
        shorter_columns = {name: len(values) for name, values in data_dict.items() if len(values) < max_length}
        if shorter_columns:
            # data_dict['velocity'] = data_dict
            # data_dict['velocity'] = [1, 1, 1, 1, 1]
            data_dict['velocity'] = np.empty(1000, dtype=float)
            try:
                for idx, distance in enumerate(data_dict['distance']):
                    if idx == 0:
                        data_dict['velocity'][idx] = 0
                        continue
                    data_dict['velocity'][idx] = data_dict['distance'][idx] / ((data_dict['timestamp'][idx] - data_dict['timestamp'][0])/1000)
            except Exception as e:
                print(e)
            # print(data_dict['velocity'][0])
            # print(data_dict['velocity'])
            data_frame = pd.DataFrame(data_dict)
            print(data_frame)
            return data_frame, attributes

        data_frame = pd.DataFrame(data_dict)
        return data_frame, attributes

all_dataframes = {}
for file_name in os.listdir('datasetsRosen'):
    if file_name.endswith('.h5'):
        file_path = os.path.join('datasetsRosen', file_name)
        frame, frame_attrs = read_hdf5(file_path)
        if frame is not None:
            all_dataframes[file_name] = (frame, frame_attrs)
print(len(all_dataframes))

     defect_channel    distance  magnetization     timestamp  velocity  \
0               0.0    0.000000      -7.944694  1.441868e+09  0.000000   
1               0.0    0.380380      -8.006990  1.441868e+09  2.861951   
2               0.0    0.760761      -7.333116  1.441868e+09  2.861951   
3               0.0    1.141141      -8.161871  1.441869e+09  2.861951   
4               0.0    1.521522      -7.449718  1.441869e+09  2.861951   
..              ...         ...            ...           ...       ...   
995             0.0  378.478478       0.641196  1.441999e+09  2.886288   
996             0.0  378.858859      -0.730467  1.441999e+09  2.886264   
997             0.0  379.239239       0.233638  1.442000e+09  2.886239   
998             0.0  379.619620       0.557410  1.442000e+09  2.886215   
999             0.0  380.000000       0.804670  1.442000e+09  2.886190   

     wall_thickness  
0         20.553395  
1         20.403704  
2         20.490569  
3         20.547338  
4

TypeError: ufunc 'divide' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [50]:
velocityMissingCount = 0
for name, (df, attrs) in all_dataframes.items():
    if not hasattr(df, 'velocity'):
       velocityMissingCount += 1
    else: 
        print(f"{name}: DataFrame Shape: {df.shape}, Attributes: {attrs}")
        print("First 5 lines of the columns:")
        # print(df[['velocity', 'defect_channel', 'distance', 'magnetization', 'timestamp', 'wall_thickness']].head())
        print(df.head())
        print("\n")
print(f'Fehlende Velocity Werte: {velocityMissingCount}')

d19dde88-83ba-45e6-b23a-e3f1412ffd55.h5: DataFrame Shape: (1000, 6), Attributes: {'configuration': 'America', 'id': 'd19dde88-83ba-45e6-b23a-e3f1412ffd55', 'instrument': 'Dog'}
First 5 lines of the columns:
   defect_channel  distance  magnetization              timestamp  velocity  \
0             0.0  0.000000     -10.473481  b'1970-11-23T08:59:1'  0.542553   
1             0.0  0.217217      -7.340053  b'1970-11-23T09:05:3'  0.501728   
2             0.0  0.434434      -6.639923  b'1970-11-23T09:12:0'  0.408769   
3             0.0  0.651652      -9.356144  b'1970-11-23T09:18:2'  0.460477   
4             0.0  0.868869     -10.811810  b'1970-11-23T09:24:4'  0.787703   

   wall_thickness  
0       20.124644  
1        7.955212  
2       12.045357  
3        8.114115  
4       21.662076  


8acbe2e4-b46c-40a2-b635-6df21f699d25.h5: DataFrame Shape: (1000, 6), Attributes: {'configuration': 'Africa', 'id': '8acbe2e4-b46c-40a2-b635-6df21f699d25', 'instrument': 'Elephant'}
First 5 lines o

# Datenbereinigung die Zweite

## Daniels kram

In [62]:
def get_dataframe_from_directory(directory):
    dataframes = {}
    for file_name in os.listdir(directory):
        if file_name.endswith('.h5'):
            file_path = os.path.join(directory, file_name)
            frame, frame_attrs = read_hdf5(file_path)
            if frame is not None:
                dataframes[file_name] = (frame, frame_attrs)
    return dataframes
    
def check_dataframe(dataframes):
    newFrame = pd.DataFrame()
    
    for frame in dataframes.values():
        numeric = drop_not_numerical_rows(frame[0])
        newFrame = newFrame.append(numeric)
    return newFrame

def drop_not_numerical_rows(dataframe):
    numeric_df = dataframe.apply(pd.to_numeric, errors='coerce')
    numeric_df = numeric_df.dropna()
    return numeric_df
      
df = check_dataframe(get_dataframe_from_directory('datasetsRosen'))

save_dataframe_to_json(df, 'generated files')

## Entfernen der Eastereggs

In [65]:
cellCount = 0

for name, (df, attrs) in all_dataframes.items():
    for index, row in df.iterrows():
        for column_name, cell_value in row.items():
            # Hier kannst du Operationen auf jeder Zelle durchführen
            cellCount += 1
            
print(f'Anzahl Zellen: {cellCount}')
cellCount2 = 0
            
for name, (df, attrs) in all_dataframes.items():
    condition = df.map(lambda cell: isinstance(cell, bytes) and cell == b'Easteregg :)')
    rows_to_drop = condition.any(axis=1)

    df.drop(df[rows_to_drop].index, inplace=True)

for name, (df, attrs) in all_dataframes.items(): 
    for index, row in df.iterrows():
        for column_name, cell_value in row.items():
            # Hier kannst du Operationen auf jeder Zelle durchführen
            cellCount2 += 1
            if cell_value == b'Easteregg :)':
                print('ALARM')
print(f'Anzahl Zellen: {cellCount2}')
print(f'Differenz: {cellCount - cellCount2}')
                
# print(f'Anzahl Zellen: {cellCount}')

Anzahl Zellen: 10132332
Anzahl Zellen: 10132332
Differenz: 0


## Datenbereinigung !!ChatGPT und kacke!!

In [ ]:
import os
import h5py
import numpy as np

def check_file(filename):
    try:
        with h5py.File(filename, 'r') as file:
            # Überprüfe hier die Struktur und Integrität der Daten
            check_dataset(file, 'defect_channel')
            check_dataset(file, 'distance')
            check_dataset(file, 'magnetization')
            check_dataset(file, 'timestamp')
            check_dataset(file, 'velocity')
            check_dataset(file, 'wall_thickness')
    except Exception as e:
        print(f"Fehler beim Lesen von {filename}: {e}")

def check_dataset(file, dataset_name):
    try:
        dataset = file['data'][dataset_name][:]
        validate_numerical_data(dataset)
        check_missing_data(dataset)
        # Weitere spezifische Überprüfungen für jedes Dataset können hinzugefügt werden
    except KeyError:
        print(f"Fehler: Dataset '{dataset_name}' nicht vorhanden.")
    except Exception as e:
        print(f"Fehler bei der Überprüfung von '{dataset_name}': {e}")

def validate_numerical_data(dataset):
    # Beispiel: Überprüfe, ob alle Werte positiv sind
    if (dataset < 0).any():
        print("Fehler: Negative Werte in den Daten.")

# Weitere Validierungsbedingungen können hier hinzugefügt werden

def check_missing_data(dataset):
    # Beispiel: Überprüfe, ob NaN-Werte in den Daten vorhanden sind
    if np.isnan(dataset).any():
        print("Fehler: NaN-Werte in den Daten.")

# Weitere Überprüfungen auf fehlende Daten können hier hinzugefügt werden

def log_result(filename, result):
    with open("log.txt", "a") as log_file:
        log_file.write(f"{filename}: {result}\n")

# Iteriere über alle Dateien und prüfe jede einzelne
folder_path = "datasetsRosen"
for file_name in os.listdir(folder_path):
    if file_name.endswith(".h5"):
        check_file(os.path.join(folder_path, file_name))


# Datenüberprüfung

## Welche Konfigurationen und Instrumente gibt es in den Datensätzen?

In [ ]:
confs_directory = 'datasetsRosen'
configurations = []
instruments = []
configAmount = 0
instAmount = 0

for idx, filename in enumerate(os.listdir(confs_directory)):
    file = h5py.File(confs_directory + '/' + filename)
    if 'data' in file:
        configName = file['data'].attrs['configuration']
        if file['data'].attrs['configuration']:
            configAmount += 1
        instrumentName = file['data'].attrs['instrument']
        if file['data'].attrs['instrument']:
            instAmount += 1
        if all(configName not in x for x in configurations):
            configurations.append(configName)
        if all(instrumentName not in x for x in instruments):
            instruments.append(instrumentName)
    if 'Daten' in file:
        configName = file['Daten'].attrs['configuration']
        if file['Daten'].attrs['configuration']:
            configAmount += 1
        instrumentName = file['Daten'].attrs['instrument']
        if file['Daten'].attrs['instrument']:
            instAmount += 1
        if all(configName not in x for x in configurations):
            configurations.append(configName)
        if all(instrumentName not in x for x in instruments):
            instruments.append(instrumentName)

print(f'Menge an Configurations: {configAmount}')
print('Configurations: ', configurations)
print(f'Menge an Instruments: {instAmount}')
print('Instruments: ', instruments)

## Wie viele falsche / fehlende Datensätze gibt es?

Prüft auf: 
    Negative Werte,
    Fehlende Datensätze / Falsche Key Bezeichnung

In [ ]:
# defect_channel, distance, magnetization, timestamp, velocity, wall_thickness
attribute = ("magnetization"
             "") # attribute ersetzen um verschiedene Attribute zu testen
count_negativeValue = 0
negativeValuesIdx = []
count_noDataset = 0
noDatasetIdx = []
eastereggCount = 0

for idx, dataset in enumerate(datasets):
    # print(dataset[attribute][...])
    if attribute in dataset:
        if b'Easteregg :)' not in dataset[attribute][...]:
            velocity_data = dataset[attribute][...].astype(float)
            if (velocity_data < 0).any():
                count_negativeValue += 1
                negativeValuesIdx.append(idx)
        else:
            eastereggCount += 1
    else:
        count_noDataset += 1
        noDatasetIdx.append(idx)
        
print(f"---------------------------------------------- {attribute} Analyse ----------------------------------------------")
print(f"Anzahl negative Werte: {count_negativeValue}\n Indizes: {negativeValuesIdx}\n")
print(f"Anzahl fehlender {attribute} Datensätze: {count_noDataset}\n Indizes: {noDatasetIdx}")
print("--------------------------------------------------------------------------------------------------------------")
print(f"Anzahl Eastereggs: {eastereggCount}")
print(datasets[1959].keys())

## Pandas Dataframe als Dateien speichern

In [52]:
def save_dataframe_to_json(dataframes, output_directory):
    json_string = dataframes.to_json(orient="table")
    file_path = os.path.join(output_directory, 'data' + ".json")

    with open(file_path, 'w') as file:
        file.write(json_string)